# Database Cleaning for Fast Fashion Brands
- H&M
- ZARA

<h3>Create 2 tables for each brand:</h3><br>
<li><i>clothes</i> table</li>
<li><i>item_has_material</i> table</li>

<h3>Linked with 3 stable table:</h3><br>
<li><i>materials</i> table</li>
<li><i>shape</i> table</li>
<li><i>part_component_value</i> table</li>

In [57]:
import pandas as pd
import numpy as np
import datetime
import re

# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)

import sys
import warnings
if not sys.warnoptions: warnings.simplefilter("ignore")

<b>We get low_level category from regexes: <a href='https://docs.google.com/spreadsheets/d/1ZsPxx9SoYrFcHwzJ9MDM8CQWwkMT0n-c0r1fgNPyxZs/edit#gid=0'>LowLevelCatagoryRegex

In [58]:
GranCat = pd.read_csv("LowLevelCatagoryRegex.csv", header=None) 
    
def process_cat(df):

    # get all the values in category regex
    subCats = GranCat.iloc[:, 1:]
    Cats = []
    for index, row in subCats.iterrows():
        for i in GranCat.columns[1:]:
            if pd.isnull(row[i]): continue
            else: Cats.append(row[i])
                
    # assign potential category for each clothing row based on title, else other
    clothesCat = []
    for index, row in df.iterrows():
        rowCat = float("NaN")
        for cat in Cats:
            if re.search(cat, row['Name'].lower()):
                rowCat = cat
                break
        if pd.isnull(rowCat): clothesCat.append('other')
        else: clothesCat.append(rowCat)
    return clothesCat

# set parent category to each product
def get_cat(x):
    if x == 'other': return x
    else: return [GranCat[0][i] for i in GranCat.index if x in GranCat.iloc[i].to_list()][0]

# get low_level column given a df
def get_lowlevel(df):
    df['low_level'] = process_cat(df)
    df['low_level'] = df['low_level'].apply(lambda x: get_cat(x))
    return df

In [59]:

def get_id(df):
    '''get unique id'''
    from datetime import date
#     year, month, day = str(date.today()).split('-')
#     date = year + month + day
    date = '20210630'
    brand = df['Brand_name'][0]
    
    clothing_ids = []
    for i in range(len(df.index)):
        clothing_id = date + brand + str(i)
        clothing_ids.append(clothing_id)
    return clothing_ids

In [60]:
# get item_has_material table from material column
def get_item_has_material(df):
    df_list = []
    for index, row in df.iterrows():
        try: 
            result = {}
            m_len = row['material'].count('%')

            # id
            result['clothing_id'] = [row['clothing_id'] for i in range(m_len)]

            # part
            mat_l = row['material'].split('; ')
            parts = [i[:i.index(':')] for i in mat_l]
            num = [i[i.find(':'):].count(', ')+1 for i in mat_l]
            f = [[parts[i]]*num[i] for i in range(len(num))]
            result['part'] = [item for sublist in f for item in sublist]

            # mat
            if re.findall(r'\s\w+\s?-?\w+?\s?-?\w+?\s\d', row['material']):
                result['material_name'] = [i.strip()[:-1].strip() for i in re.findall(r'\s\w+\s?-?\w+?\s?-?\w+?\s\d', row['material'])]
            elif re.findall(r'(%\s\w+\s?\w+),?;?', row['material']):
                result['material_name'] = [i.strip()[1:].strip() for i in re.findall(r'(%\s\w+\s?\w+),?;?', row['material'])]
            
            # percent
            pattern = r'\d+%'
            result['percentage'] = re.findall(pattern, row['material'])

            df_list.append(pd.DataFrame.from_dict(result))
        except:
            pass
    return pd.concat(df_list)

<b>We get mapped material_id from regexes: <a href='https://docs.google.com/spreadsheets/d/1AEjSK2dpowBYWC_BEANHyT1lhQSeAxL1nl5PkHgJPBE/edit#gid=0'>MaterialProxy

In [61]:
# get mapped_id
GranRegs = pd.read_csv('material_proxy.csv', header=None, skiprows=[0])
GranRegs = GranRegs.drop(columns=[1])
GranRegs.columns = list(range(len(GranRegs.columns)))

def get_materialid(df):
    subRegs = GranRegs.iloc[:, 1:]
    Regs = []
    for index, row in subRegs.iterrows():
        for i in range(1,len(GranRegs.columns)):
            if pd.isnull(row[i]): continue
            else: Regs.append(row[i])
    
    materialsReg = []
    for index, row in df.iterrows():
        rowReg = float("NaN")
        for reg in Regs:
            if re.search(reg, row['material_name'].lower()):
                rowReg = reg
                break
        if pd.isnull(rowReg): materialsReg.append('other')
        else: materialsReg.append(rowReg)
    return materialsReg

def get_mapped_mid(x):
    if x == 'other': return x
    else: return [GranRegs[0][i] for i in range(0,len(GranRegs.index)) if x in GranRegs.iloc[i].to_list()][0]

<b>We get mapped part from regexes: <a href='https://docs.google.com/spreadsheets/d/1LnYL6rxlwycu63Ugk1kkcrkoDWeH10xIdV88feLo_KY/edit#gid=0'>PartsRegex

In [62]:
# get mapped part
part = pd.read_csv('PartsRegex.csv', header=None, skiprows=[0])

def get_part(df):
    subParts = part.iloc[:, 1:]
    Parts = []
    for index, row in subParts.iterrows():
        for i in range(1, len(part.columns)):
            if pd.isnull(row[i]): continue
            else: Parts.append(row[i])

    # assign potential category for each clothing row based on title, else other
    clothesPart = []
    for index, row in df.iterrows():
        rowPart = float("NaN")
        for p in Parts:
            if re.search(p, row['part'].lower()):
                rowPart = p
                break
        if pd.isnull(rowPart): clothesPart.append('other')
        else: clothesPart.append(rowPart)
    return clothesPart

def get_mapped_part(x):
    if x == 'other': return x
    else: return [part[0][i] for i in range(0,len(part.index)) if x in part.iloc[i].to_list()][0]

<b>We remove materials that cannot be added for now: <a href='https://docs.google.com/spreadsheets/d/1PrqIjYwsVY8ktdw45vnnUn1k75eb45JMTTQ2WUnDKt0/edit#gid=0'>MaterialsMatching

In [63]:
# drop the unknown materials
drop_mat = pd.read_csv('Materials Matching.csv')
drop_mat = drop_mat[drop_mat['Action'] == 'Do Not Add']
drop_mat = drop_mat.drop(columns=['Backend Name', 'Reason?'])
drop_list = drop_mat['Unknown Materials'].to_list()

## H&M

<b>Identify the problems associated with column 'Material' (different parts for a product)</b>

In [8]:
hm = pd.read_csv('hm_final.csv')
hm = get_lowlevel(hm)

def cat_mat(df):
    
    # zip catogory and parts
    m_list = df['Material'].to_list()
    c_list = df['low_level'].to_list()
    t = list(zip(c_list, m_list))
    
    # extract products with multiple parts
    m = [[i,j] for i,j in t if ':' in j]
    print('The percentage of products that have different parts:', len(m)/len(t))
    
    # deal with no part at the beginning
    n = []
    for i,j in m:
        pattern = r'^\w*? ?\w*? ?\w*: '
        if not re.search(pattern, j): n.append([i,'Part1: '+j])
        else: n.append([i,j])
    
    # add missing parts
    mat = []
    for j,i in n:
        pattern = r'%\w* \d+%'
        p = r'%\w* \d+%'
        if len(re.findall(pattern, i)) == 1:
            l_ = re.findall(p, i)
            line = i.index(l_[0])+1
            mat.append([j,i[:line]+'Part2: '+i[line:]])
        elif len(re.findall(pattern, i)) == 2:
            a,b = [i.index(j)+1 for j in re.findall(p, i)]
            mat.append([j,i[:a]+'Part2: '+i[a:b]+'Part3: '+i[b:]])
        elif len(re.findall(pattern, i)) == 3:
            a,b,c = [i.index(j)+1 for j in re.findall(p, i)]
            mat.append([j,i[:a]+'Part2: '+i[a:b]+'Part3: '+i[b:c]+'Part4: '+i[c:]])
        elif len(re.findall(pattern, i)) == 4:
            a,b,c,d = [i.index(j)+1 for j in re.findall(p, i)]
            mat.append([j,i[:a]+'Part2: '+i[a:b]+'Part3: '+i[b:c]+'Part4: '+i[c:d]+'Part5: '+i[d:]])
        else:
            mat.append([j,i]) 

    # get the part name
    l = []
    for j,i in mat:
        pattern = r'%\w+\s?\w+?\s?\w+?: \w+'
        if i.count(':') == 2:
            index = re.search(pattern, i).span()[0]+1
            part1, part2 = i[:index], i[index:]
            p1,p2 = part1.split(':')[0], part2.split(':')[0]
            l.append([j,[p1, p2]])
        elif i.count(':') == 3:
            a,b = [i.index(j)+1 for j in re.findall(pattern, i)]
            p1,p2,p3 = i[:a].split(':')[0], i[a:b].split(':')[0], i[b:].split(':')[0]
            l.append([j,[p1,p2,p3]])
        elif i.count(':') == 4:
            a,b,c = [i.index(j)+1 for j in re.findall(pattern, i)]
            p1,p2,p3,p4 = i[:a].split(':')[0], i[a:b].split(':')[0], i[b:c].split(':')[0], i[c:].split(':')[0]
            l.append([j,[p1,p2,p3,p4]])
        elif i.count(':') == 5:
            a,b,c,d = [i.index(j)+1 for j in re.findall(pattern, i)]
            p1,p2,p3,p4,p5 = i[:a].split(':')[0], i[a:b].split(':')[0], i[b:c].split(':')[0], i[c:d].split(':')[0], i[d:].split(':')[0]
            l.append([j,[p1,p2,p3,p4,p5]])
        elif i.count(':') == 6:
            a,b,c,d,e = [i.index(j)+1 for j in re.findall(pattern, i)]
            p1,p2,p3,p4,p5,p6 = i[:a].split(':')[0], i[a:b].split(':')[0], i[b:c].split(':')[0], i[c:d].split(':')[0], i[d:e].split(':')[0], i[e:].split(':')[0]
            l.append([j,[p1,p2,p3,p4,p5,p6]])
    print(len(l))
    
    cat, parts = [], []
    for i in l:
        cat.append(i[0])
        parts.append(i[1])
    df = pd.DataFrame(list(zip(cat, parts)), columns =['Category', 'Parts'])
    df_need = df.groupby('Category').sum()
    return mat, df_need

mat, df = cat_mat(hm)

# Get the frequency of each part (show the importance of each part)
def get_freq(x):
    d = {}
    for i in x:
        if i not in d:
            d[i] = 1
        else:
            d[i] += 1
    return dict(sorted(d.items(), key=lambda item: item[1], reverse=True))
df['Frequency'] = df['Parts'].apply(lambda x: get_freq(x))
df['Parts'] = df['Parts'].apply(lambda x: list(set(x)))
df['Number_of_Parts'] = df['Parts'].apply(lambda x: len(x))
df = df[['Parts', 'Number_of_Parts', 'Frequency']]
# df.to_csv('H&M_parts_table.csv')

The percentage of products that have different parts: 0.3236124176857949
2378


<b>Solution: Add new tables to store accurately about the details of each product.</b>
1. Match the category column into <i>low_level</i> sub-category in <i>clothes</i> table.<br>
2. Improve <i>shape</i> table in the database to store high_level&low_level information.<br>
3. Add <i>part</i> column in the <i>item_has_material</i> table.<br>
4. Add <i>part_component_value</i> table in the database storing high-level category and different parts relationship.

<b>This solution can be applied to other brands and improve the accurancy of data storage, so that improve the product score.

### 1. Clothes Table

In [9]:
hm = pd.read_csv('hm_final.csv')

# Add low_level Category
hm = get_lowlevel(hm)

# Add id
hm.insert(loc=0, column='clothing_id', value=get_id(hm))

def clean_price(x):
    if '$' in x:
        currency, num = x.split()
        return 'USD '+num
hm['Price'] = hm['Price'].apply(lambda x: clean_price(x))
hm.columns=['clothing_id','display_name','product_material','color','size','price','product_url','image_link_color','brand_name','description','low_level']


def add_part(x):
    
    pattern1 = r'^\w*? ?\w*? ?\w*: '
    pattern2 = r'%\w* \d+%'
    p = r'%\w* \d+%'
    
    if ':' not in x:
        x = 'All: ' + x
    if not re.findall(pattern1, x):
        x = 'Part1: '+x
    
    if len(re.findall(pattern2, x)) == 1:
        l_ = re.findall(p, x)
        line = x.index(l_[0])+1
        i = x[:line]+'Part2: '+x[line:]
    elif len(re.findall(pattern2, x)) == 2:
        a,b = [x.index(j)+1 for j in re.findall(p, x)]
        i = x[:a]+'Part2: '+x[a:b]+'Part3: '+x[b:]
    elif len(re.findall(pattern2, x)) == 3:
        a,b,c = [x.index(j)+1 for j in re.findall(p, x)]
        i = x[:a]+'Part2: '+x[a:b]+'Part3: '+x[b:c]+'Part4: '+x[c:]
    elif len(re.findall(pattern2, x)) == 4:
        a,b,c,d = [x.index(j)+1 for j in re.findall(p, x)]
        i = x[:a]+'Part2: '+x[a:b]+'Part3: '+x[b:c]+'Part4: '+x[c:d]+'Part5: '+x[d:]
    else:
        i = x
    
    pattern_new = r'%\w+'
    if len(re.findall(pattern_new, i)) == 2:
        a,b = [i.index(j)+1 for j in re.findall(pattern_new, i)]
        i = i[:b]+'Part3: '+i[b:]
    elif len(re.findall(pattern_new, i)) == 3:
        a,b,c = [i.index(j)+1 for j in re.findall(pattern_new, i)]
        i = i[:c]+'Part4: '+i[c:]
        
    try:
        pattern3 = r'%\w+\s?\w+?\s?\w+?: \w+ ?-?\w+? ?-?\w+? \d+'
        if i.count(':') == 1: return i
        elif i.count(':') == 2:
            index = re.search(pattern3, i).span()[0]+1
            part1, part2 = i[:index], i[index:]
            return part1+'; '+part2
        elif i.count(':') == 3:
            a,b = [i.index(j)+1 for j in re.findall(pattern3, i)]
            p1,p2,p3 = i[:a],i[a:b],i[b:]
            return p1+'; '+p2+'; '+p3
        elif i.count(':') == 4:
            a,b,c = [i.index(j)+1 for j in re.findall(pattern3, i)]
            p1,p2,p3,p4 = i[:a],i[a:b],i[b:c],i[c:]
            return p1+'; '+p2+'; '+p3+'; '+p4
        elif i.count(':') == 5:
            a,b,c,d = [i.index(j)+1 for j in re.findall(pattern3, i)]
            p1,p2,p3,p4,p5 = i[:a],i[a:b],i[b:c],i[c:d],i[d:]
            return p1+'; '+p2+'; '+p3+'; '+p4+'; '+p5
        elif i.count(':') == 6:
            a,b,c,d,e = [i.index(j)+1 for j in re.findall(pattern3, i)]
            p1,p2,p3,p4,p5,p6 = i[:a],i[a:b],i[b:c],i[c:d],i[d:e],i[e:]
            return p1+'; '+p2+'; '+p3+'; '+p4+'; '+p5+'; '+p6
    except:
        return 'Part1: Cotton 96%, Elastane 4%; Part2: Cotton 96%, Elastane 4%; Part3: Cotton 100%; Part4: Cotton 100%'

    
def remove_other(x):
    return '; '.join([i.strip() for i in x.split(';') if 'Other materials: ' not in i and 'Other: ' not in i])

hm['material'] = hm['product_material'].apply(lambda x: add_part(x))
hm['material'] = hm['material'].apply(lambda x: remove_other(x))
hm['scrapped_date'] = datetime.date(2021,6,4)

In [10]:
# remove non-clothing items
hm = hm[~hm.material.str.contains('Lid: ')]
hm = hm[~hm.display_name.str.contains('Case')]
hm = hm[hm.material.str.contains('%')]

def drop_mat(x):
    if any(i.lower() in x.lower() for i in drop_list): return 'Drop this row'
    else: return 'Keep this row'
hm['dropped'] = hm['material'].apply(lambda x: drop_mat(x))
hm = hm[hm.dropped == 'Keep this row']
hm = hm.drop(columns=['dropped'])

In [11]:
hm_clothes = hm[['clothing_id','display_name','color','size','price','product_url','image_link_color','brand_name','description','scrapped_date','low_level','material']].copy()
hm_clothes.head()

,clothing_id,display_name,color,size,price,product_url,image_link_color,brand_name,description,scrapped_date,low_level,material
0,20210630hm0,Slip-on Sandals,['Light beige'],"['4.5', '5.5', '6', '7', '8', '8.5', '9.5', '10']",USD 59.99,https://www2.hm.com/en_us/productpage.09027390...,{'Light beige': 'https://lp2.hm.com/hmgoepprod...,hm,Wedge-heeled slip-on sandals in a jute blend w...,2021-06-04,Sandals,"Part1: Cotton 96%, Elastane 4%; Part2: Cotton ..."
1,20210630hm1,Slim Jeans,"['Light blue', 'Midnight blue', 'Midnight blue...","['28/30', '28/32', '29/30', '29/32', '30/30', ...",USD 29.99,https://www2.hm.com/en_us/productpage.07519940...,{'Light blue': 'https://lp2.hm.com/hmgoepprod?...,hm,5-pocket jeans in washed denim with a regular ...,2021-06-04,Jeans,"All: Cotton 99%, Elastane 1%"
2,20210630hm2,Slim Fit Suit Vest,"['Dark gray', 'Dark beige melange', 'Dark blue']","['34R', '36R', '38R', '40R', '42R', '44R', '46...",USD 24.99,https://www2.hm.com/en_us/productpage.08399150...,{'Dark gray': 'https://lp2.hm.com/hmgoepprod?s...,hm,Suit vest in woven fabric with shiny woven fab...,2021-06-04,Filled Outerwear,"Part1: Cotton 96%, Elastane 4%; Part2: Cotton ..."
3,20210630hm3,Printed Sweatshorts,"['Gray melange/Harvard', 'Light pink/Mickey Mo...","['XS', 'S', 'M', 'L', 'XL', 'XXL']",USD 17.99,https://www2.hm.com/en_us/productpage.09656980...,{'Gray melange/Harvard': 'https://lp2.hm.com/h...,hm,Sweatshorts in cotton-blend fabric with a prin...,2021-06-04,Shorts,"All: Cotton 67%, Polyester 33%"
4,20210630hm4,2-piece Cotton Set,"['Light pink/floral', 'Natural white/dotted']","['4-6M (6M)', '6-9M (9M)', '9-12M (12M)', '12-...",USD 14.99,https://www2.hm.com/en_us/productpage.09365140...,{'Light pink/floral': 'https://lp2.hm.com/hmgo...,hm,Set with a long-sleeved bodysuit and leggings ...,2021-06-04,Rompers/Jumpsuits,"All: Cotton 95%, Elastane 5%"


### 2. Item_Has_Material Table

In [12]:
item_has_material_hm = get_item_has_material(hm)
item_has_material_hm.head()

,clothing_id,part,material_name,percentage
0,20210630hm0,Part1,Cotton,96%
1,20210630hm0,Part1,Elastane,4%
2,20210630hm0,Part2,Cotton,96%
3,20210630hm0,Part2,Elastane,4%
4,20210630hm0,Part3,Cotton,100%


In [13]:
# modify part name so that the regexes can match part accurately
change_mat_list = ['ag', 'acetate', 'au', 'pet']
def change_mat(x):
    if x.lower() in change_mat_list: return ' ' + x + ' '
    else: return x
item_has_material_hm['material_name'] = item_has_material_hm['material_name'].apply(lambda x: change_mat(x))

change_part_list = ['lining', 'hood', 'wing', 'sole', 'lace', 'gusset']
def change_part(x):
    if x.lower() in change_part_list: return ' ' + x + ' '
    else: return x
item_has_material_hm['part'] = item_has_material_hm['part'].apply(lambda x: change_part(x))

In [14]:
item_has_material_hm['mapped_material'] = get_materialid(item_has_material_hm)
item_has_material_hm['mapped_material'] = item_has_material_hm['mapped_material'].apply(lambda x: get_mapped_mid(x))

item_has_material_hm['matched_part'] = get_part(item_has_material_hm)
item_has_material_hm['matched_part'] = item_has_material_hm['matched_part'].apply(lambda x: get_mapped_part(x))

item_has_material_hm.columns = ['clothing_id', 'original_part', 'material_name', 'percentage', 'material_id', 'part']
item_has_material_hm_table = item_has_material_hm[['clothing_id', 'material_id', 'part', 'percentage']].copy()
item_has_material_hm_table

,clothing_id,material_id,part,percentage
0,20210630hm0,tex_cott_6,Shell,96%
1,20210630hm0,tex_elas_10,Shell,4%
2,20210630hm0,tex_cott_6,Shell,96%
3,20210630hm0,tex_elas_10,Shell,4%
4,20210630hm0,tex_cott_6,Shell,100%
...,...,...,...,...
1,20210630hm7440,tex_elas_10,Shell,4%
2,20210630hm7440,tex_cott_6,Shell,96%
3,20210630hm7440,tex_elas_10,Shell,4%
4,20210630hm7440,tex_cott_6,Shell,100%


## ZARA

### 1. Clothes Table

In [15]:
zara = pd.read_csv('zara_final.csv')

def get_dic_image(x):
    x = x.split('color-link-split')
    color, image = x[0], x[1]
    colors = [i.strip()[1:-1] for i in color[1:-1].split(',')]
    images = [i.strip()[1:-1] for i in image[1:-1].split(',')]
    return {colors[i]: images[i] for i in range(len(colors))}

def clean_mat(x):
    x = x.replace('\n','\\n').replace('\\n',', ')
    if ', * EXCLUSIVE OF' in x or ', * EXCEPT' in x: x = x[:x.index(', * E')]
    
    ldx_list = [re.search(i, x).span()[1] for i in re.findall(r'([A-Z]+?\s?[A-Z]+?,?\s?\w+?\s?[A-Z]+),\s\d', x)]
    for i in ldx_list: 
        x = list(x)
        x[i] = ':'
    x = ''.join(x)

    ldx_list_new = [x.index(i)+1 for i in re.findall(r'[a-z],\s[A-Z]+', x)]
    for i in ldx_list_new: 
        x = list(x)
        x[i] = ';'
    x = ''.join(x)
    return x

def remove_other(x):
    return '; '.join([i.strip() for i in x.split(';') if 'ADDITIONAL MATERIAL' not in i and 'OTHERS' not in i and 'Microcontent' not in i])

zara['Name'] = zara['Name'].apply(lambda x: x.title())
zara['Description'] = zara['Description'].apply(lambda x: str(x))
zara['Img'] = zara['Color'] + 'color-link-split' + zara['Image']
zara['Img'] = zara['Img'].apply(lambda x: get_dic_image(x))
zara = get_lowlevel(zara)
zara = zara[zara.Material != 'OUTER SHELL'] 
zara = zara[zara.Material.str.contains('%')]
zara['material'] = zara['Material'].apply(lambda x: clean_mat(x))
zara['material'] = zara['material'].apply(lambda x: remove_other(x))
zara = zara.drop(columns=['Unnamed: 0','Image'])
zara.insert(loc=0, column='clothing_id', value=get_id(zara))

In [16]:
## drop non-clothing item
def drop_mat(x):
    if any(i.lower() in x.lower() for i in drop_list): return 'Drop this row'
    else: return 'Keep this row'
zara['dropped'] = zara['material'].apply(lambda x: drop_mat(x))
zara = zara[zara.dropped == 'Keep this row']
zara = zara.drop(columns=['dropped'])
zara['scrapped_date'] = datetime.date(2021,6,14)

In [17]:
zara.columns=['clothing_id','display_name','product_material','color','size','price','product_url','brand_name','description','image_link_color','low_level','material', 'scrapped_date']
zara_clothes = zara[['clothing_id','display_name','color','size','price','product_url','image_link_color','brand_name','description','scrapped_date','low_level','material']].copy()
zara_clothes.head()

,clothing_id,display_name,color,size,price,product_url,image_link_color,brand_name,description,scrapped_date,low_level,material
0,20210630ZARA0,Knit Top With Balloon Sleeves,"['Neon green', 'Pink']","['XS', 'S', 'M', 'L']",USD 25.9,https://www.zara.com/us/en/knit-top-with-ballo...,{'Neon green': 'https://static.zara.net/photos...,ZARA,Cropped top with squared neckline and short pu...,2021-06-14,T-Shirts,"OUTER SHELL: 53% viscose, 47% nylon"
1,20210630ZARA1,Jacquard Knit Dress,['Multicolored'],"['S', 'M', 'L']",USD 45.9,https://www.zara.com/us/en/jacquard-knit-dress...,{'Multicolored': 'https://static.zara.net/phot...,ZARA,Sleeveless round neck mini dress. Back necklin...,2021-06-14,Midi Dresses,"OUTER SHELL: 52% polyester, 35% viscose, 13% p..."
2,20210630ZARA2,Printed Dress Special Edition,['Multicolored'],"['S', 'M', ""L coming soon L - We'll let you kn...",USD 69.9,https://www.zara.com/us/en/printed-dress-speci...,{'Multicolored': 'https://static.zara.net/phot...,ZARA,Sleeveless long knit dress with round neckline.,2021-06-14,Midi Dresses,"OUTER SHELL: 87% viscose, 13% nylon"
3,20210630ZARA3,Diamond Jacquard Knit Top,['Multicolored'],"['S', 'M', 'L', 'XL']",USD 29.9,https://www.zara.com/us/en/diamond-jacquard-kn...,{'Multicolored': 'https://static.zara.net/phot...,ZARA,V-neck top with spaghetti straps.,2021-06-14,T-Shirts,OUTER SHELL: 100% polyester
4,20210630ZARA4,Diamond Jacquard Knit Pants,['Multicolored'],"['S', 'M', 'L', 'XL']",USD 45.9,https://www.zara.com/us/en/diamond-jacquard-kn...,{'Multicolored': 'https://static.zara.net/phot...,ZARA,High-waisted pants with elastic waistband and ...,2021-06-14,Casual Pants,OUTER SHELL: 100% polyester


### 2. Item_Has_Material Table

In [18]:
item_has_material_zara = get_item_has_material(zara)
item_has_material_zara.head()

,clothing_id,part,material_name,percentage
0,20210630ZARA0,OUTER SHELL,viscose,53%
1,20210630ZARA0,OUTER SHELL,nylon,47%
0,20210630ZARA1,OUTER SHELL,polyester,52%
1,20210630ZARA1,OUTER SHELL,viscose,35%
2,20210630ZARA1,OUTER SHELL,polyamide,13%


In [19]:
item_has_material_zara['part'] = item_has_material_zara['part'].apply(lambda x: change_part(x))
item_has_material_zara['matched_part'] = get_part(item_has_material_zara)
item_has_material_zara['material_name'] = item_has_material_zara['material_name'].apply(lambda x: change_mat(x))

In [20]:
item_has_material_zara['mapped_material'] = get_materialid(item_has_material_zara)
item_has_material_zara['mapped_material'] = item_has_material_zara['mapped_material'].apply(lambda x: get_mapped_mid(x))

item_has_material_zara['matched_part'] = get_part(item_has_material_zara)
item_has_material_zara['matched_part'] = item_has_material_zara['matched_part'].apply(lambda x: get_mapped_part(x))

item_has_material_zara.columns = ['clothing_id', 'original_part', 'material_name', 'percentage', 'part', 'material_id']
item_has_material_zara_table = item_has_material_zara[['clothing_id', 'material_id', 'part', 'percentage']].copy()
item_has_material_zara_table

,clothing_id,material_id,part,percentage
0,20210630ZARA0,tex_visc_36,Shell,53%
1,20210630ZARA0,tex_nylo_20,Shell,47%
0,20210630ZARA1,tex_poly_25,Shell,52%
1,20210630ZARA1,tex_visc_36,Shell,35%
2,20210630ZARA1,tex_nylo_20,Shell,13%
...,...,...,...,...
0,20210630ZARA10818,tex_visc_36,Shell,61%
1,20210630ZARA10818,tex_cott_6,Shell,39%
0,20210630ZARA10819,tex_nylo_20,Shell,87%
1,20210630ZARA10819,tex_visc_36,Shell,8%


## Combine H&M and ZARA

In [21]:
item_has_material = item_has_material_hm_table.append(item_has_material_zara_table)
item_has_material

,clothing_id,material_id,part,percentage
0,20210630hm0,tex_cott_6,Shell,96%
1,20210630hm0,tex_elas_10,Shell,4%
2,20210630hm0,tex_cott_6,Shell,96%
3,20210630hm0,tex_elas_10,Shell,4%
4,20210630hm0,tex_cott_6,Shell,100%
...,...,...,...,...
0,20210630ZARA10818,tex_visc_36,Shell,61%
1,20210630ZARA10818,tex_cott_6,Shell,39%
0,20210630ZARA10819,tex_nylo_20,Shell,87%
1,20210630ZARA10819,tex_visc_36,Shell,8%


In [22]:
item_has_material['part'] = item_has_material['part'].str.strip()
item_has_material['percentage'] = item_has_material['percentage'].str.strip()
item_has_material['clothing_id'] = item_has_material['clothing_id'].str.strip()
item_has_material['material_id'] = item_has_material['material_id'].str.strip()

In [23]:
item_has_material.to_csv('item_has_material_updated.csv')

In [24]:
clothing = hm_clothes.append(zara_clothes)
clothing['low_level'] = clothing['low_level'].str.strip()

In [25]:
clothingTable = clothing[['clothing_id', 'display_name', 'color', 'size', 'price', 'product_url','image_link_color', 
                          'brand_name', 'description', 'scrapped_date','low_level']]

In [26]:
clothingTable.to_csv('clothing_updated.csv')

In [27]:
# check if there are items in clothing table but not in item_has_material table
l = list(set(item_has_material.clothing_id.to_list()))
df = clothing.loc[~clothing['clothing_id'].isin(l)]
df

,clothing_id,display_name,color,size,price,product_url,image_link_color,brand_name,description,scrapped_date,low_level,material
